<a href="https://colab.research.google.com/github/willianszwy/Aprendizado-Supervisionado-no-Neuronio-Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

Seed para reproducibilidade

In [ ]:
random.seed(323452345)

In [ ]:

exemplos = np.fromfile('dataAll.txt').reshape(-1,3)


In [ ]:
class Perceptron:

    def __init__(self, tamanho, bias = 0, taxa_aprendizado= 0.1) -> None:
        self.bias = bias
        self.taxa_aprendizado = taxa_aprendizado
        self.pesos = np.zeros(tamanho + 1)
        self.pesos[0] = random.uniform(-0.5,0.5)
        self.pesos[1] = random.uniform(-0.5,0.5)
        self.pesos[2] = random.uniform(-0.5,0.5)
        self.entradas = np.empty(tamanho + 1)
        self.entradas[0] = self.bias
        self.erro = 0
        self.atualiza_pesos_contador = 0 


    def treinar(self, d):
        epoca = 0
        continua = True
        while continua:
            epoca += 1
            print("Época: ", epoca)
            contem_erro = False
            qtd_ajustes_pesos_epoca = 0

            for i in range(d.shape[0]):

                self.entradas[1] = d[i][0]
                self.entradas[2] = d[i][1]

                y = self.predicao()
                self.calcula_erro(y, d[i][2])

                if self.erro != 0:
                    self.atualiza_pesos()
                    qtd_ajustes_pesos_epoca += 1
                    contem_erro = True      

            print("Total ajustes de pesos da época: ", qtd_ajustes_pesos)

            if not contem_erro:
                continua = False
        
        print("Número total de ajustes dos pesos: ", self.atualiza_pesos_contador)
        print("Número de Épocas: ", epoca)

    def funcao_de_ativacao(self, x):
        return 1 if x >= 0 else 0

    def predicao(self):
        y = self.pesos.T.dot(self.entradas)
        return self.funcao_de_ativacao(y)

    def calcula_erro(self,y, yd):
        self.erro = yd - y

    def delta(self):
        return self.taxa_aprendizado * self.erro * self.entradas

    def atualiza_pesos(self):
        
            self.pesos =  self.pesos + self.delta()
            self.atualiza_pesos_contador += 1
            print("pesos atualizados: ",self.pesos)

In [ ]:
neuronio = Perceptron(2)
neuronio.treinar(exemplos)

Época:  1
pesos atualizados:  [ -0.23597521  36.21398462 -24.30116417]
pesos atualizados:  [  -0.23597521  -17.44506538 -100.79691417]
pesos atualizados:  [ -0.23597521  55.35095462 -64.96735417]
pesos atualizados:  [ -0.23597521 137.94965462  14.88847583]
pesos atualizados:  [ -0.23597521 143.79541462 -63.40611417]
pesos atualizados:  [  -0.23597521  103.78650462 -141.29880417]
pesos atualizados:  [ -0.23597521 157.45631462 -92.20653417]
pesos atualizados:  [  -0.23597521  104.49625462 -149.07599417]
pesos atualizados:  [ -0.23597521 164.05577462 -98.95366417]
pesos atualizados:  [  -0.23597521  118.53114462 -164.58959417]
pesos atualizados:  [ -0.23597521 209.24700462 -89.87627417]
pesos atualizados:  [  -0.23597521  151.46601462 -150.18442417]
pesos atualizados:  [  -0.23597521  103.19945462 -198.65414417]
pesos atualizados:  [  -0.23597521  173.64680462 -156.53787417]
pesos atualizados:  [  -0.23597521  126.80480462 -207.94774417]
pesos atualizados:  [  -0.23597521  196.66843462 -1